<a href="https://colab.research.google.com/github/dina-lab3D/MHCfold/blob/main/MHCfold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dina-lab3D/MHCfold/blob/main/MHCfold.ipynb)


#MHCfold
MHCfold is a novel deep learning-based end-to-end modeling tool that given a MHC allele and peptide sequences, directly produces the 3D coordinates of the Cβ atoms, aswell the binary specificity prediction .

The source code and the trained model can be found @ https://github.com/dina-lab3D/MHCfold

**Structure module**

<br>
<img src=https://drive.google.com/uc?id=1hLLQUrOsUCi8t-GtUmYfzgZLMnb6VKpx
width="700">


**Classification module**

<br>
<img src=https://drive.google.com/uc?id=15IekywPLKHM-DByMxs8eAjvD0O_Nbzbj width="700">


<strong>For Citation use: </strong> [NanoNet paper](https://www.frontiersin.org/articles/10.3389/fimmu.2022.958584/full)


In [ ]:
#@title Input MHC sequence/MHC fasta file (creates a MHC structure for each entry in the fasta file)
from google.colab import files
import re
import os
from IPython.display import clear_output
#@markdown ### Input Output
#@markdown ##### * Alpha chain - either the sequence, or a fasta file. 
#@markdown ##### fasta file needs to end with '.fa', and should be formatted: alpha_chain_sq:beta_chain_seq:peptide_seq

#@markdown ##### * Beta chain - you can leave empty, and a default sequence will be used
#@markdown ---
# input_type = 'Fasta file' #@param ["Sequence (String)", "Fasta file"]
input_chain_alpha = 'MHCfold/test_fasta.fa' #@param {type:"string"}
input_chain_beta = '' #@param {type:"string"}
if not input_chain_beta:
  input_chain_beta = "IQRTPKIQVYSRHPAENGKSNFLNCYVSGFHPSDIEVDLLKNGERIEKVEHSDLSFSKDWSFYLLYYTEFTPTEKDEYACRVNHVTLSQPKIVKWDRDM"
input_chain_peptide = 'TPYDINQML' #@param {type:"string"}
prediction_output = 'Both' #@param ["Structure", "Binary classification", "Both"]
output_dir = 'MHCNetResults' #@param {type:"string"}
input_type = 'Fasta file' if input_chain_alpha.endswith(".fa") else 'Sequence'
# remove whitespaces
output_dir = "".join(output_dir.split())
output_dir = re.sub(r'\W+', '', output_dir)

if '.' in input_chain_alpha and not input_chain_alpha.endswith('.fa'):
  raise Exception("fasta file needs to end with '.fa'")

#@markdown ---
#@markdown ### Advanced settings
# write_into_single_pdb_file = False #@param {type:"boolean"}
reconsrtuct_side_chains_using_modeller = False #@param {type:"boolean"}
modeller_license_key = '' #@param {type:"string"}
visualize_results = True #@param {type:"boolean"}
# reconsrtuct_side_chains_using_scwrl = "" #@param {type:"string"}
##@markdown  (insert Scwrl4 executable path)
# reconsrtuct_side_chains_using_modeller = False
reconsrtuct_side_chains_using_scwrl = False

#@markdown ---
#@markdown ### Saving options
save_to_google_drive = False #@param {type:"boolean"}
#@markdown ---
#@markdown  To run MHCNet hit `Runtime` -> `Run all`

if reconsrtuct_side_chains_using_modeller and write_into_single_pdb_file:
  raise ValueError("Can't reconstruct side chains with single_file option select only one option from 'write_into_single_pdb_file' and 'reconsrtuct_side_chains_using_modeller'")
if reconsrtuct_side_chains_using_modeller and modeller_license_key == '':
  raise ValueError("Please insert a valid license key!, you can get one from here: https://salilab.org/modeller/registration.html")

input_mhc = input_chain_alpha

if input_type == 'Sequence':  
  # remove whitespaces
  input_mhc = input_chain_alpha + ":" + input_chain_beta + ":" + input_chain_peptide
  input_mhc = "".join(input_mhc.split())
  input_mhc = re.sub(r'[^a-zA-Z:]','', input_mhc).upper()
  with open("nb_fasta.fa", "w") as fa_file:
    fa_file.write("> User_sequence\n")
    fa_file.write("{}\n".format(input_mhc))
  input_mhc = "nb_fasta.fa"

if not os.path.exists(input_mhc):
  raise ValueError("can't find fasta file {}.".format(input_mhc))

if save_to_google_drive == True:
  from pydrive.drive import GoogleDrive
  from pydrive.auth import GoogleAuth
  from google.colab import auth
  from oauth2client.client import GoogleCredentials
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  print("Saving results into Drive")


In [ ]:
#@title Clone MHCNet trained model 


%%bash

if [ ! -f MHCNetReady ]; then
  # install dependencies
  pip -q install biopython
  pip install py3Dmol
  touch MHCNetReady
fi
  # download model
if [ -d "MHCfold/" ]; then
  rm -r MHCfold/
fi
git clone https://github.com/alonson23/MHCfold --quiet
unzip MHCfold/v7_date_5_9_2022.zip -d MHCfold/
  

Archive:  MHCfold/v7_date_5_9_2022.zip
   creating: MHCfold/v7_date_5_9_2022/assets/
  inflating: MHCfold/v7_date_5_9_2022/keras_metadata.pb  
  inflating: MHCfold/v7_date_5_9_2022/saved_model.pb  
   creating: MHCfold/v7_date_5_9_2022/variables/
  inflating: MHCfold/v7_date_5_9_2022/variables/variables.data-00000-of-00001  
  inflating: MHCfold/v7_date_5_9_2022/variables/variables.index  


In [ ]:
#@title Download Modeller
#@markdown If 'reconsrtuct_side_chains_using_modeller' is set to false, you can skip this step.
if not os.path.exists("ModellerReady") and reconsrtuct_side_chains_using_modeller:
  #@markdown **You can get a license key for Modeller from** **[here](https://salilab.org/modeller/registration.html)** .
  # modeller_license_key = '' #@param {type:"string"}
  #MODELIRANJE
  !wget https://salilab.org/modeller/10.1/modeller-10.1.tar.gz
  !tar -zxf modeller-10.1.tar.gz
  print("MODELLER extraction completed")
  %cd modeller-10.1
  #And we prepare a file containing the minimal setup elements
  #For installing, including a license key
  with open('modeller_config', 'a') as f:
    f.write("3\n")
    f.write("/content/compiled/MODELLER\n")
  #ADD YOUR LICENSE KEY HERE!
    f.write(f"{modeller_license_key}\n")
  !./Install < modeller_config
  print("MODELLER set up completed")

  %cd /content/
  #Creating a symbolic link
  %cd modeller-10.1
  !ln -sf /content/compiled/MODELLER/bin/mod10.1 /usr/bin/
  %cd /content/
  #Checking if MODELLER works
  !mod10.1 | awk 'NR==1{if($1=="usage:") print "MODELLER succesfully installed"; else if($1!="usage:") print "Something went wrong. Please install again"}'

  with open("/content/compiled/MODELLER/modlib/modeller/config.py", "r") as file:
    lines = file.readlines()

  with open("/content/compiled/MODELLER/modlib/modeller/config.py", "w") as file:
    file.write(lines[0])
    file.write(f"license = '{modeller_license_key}'\n")
  with open("ModellerReady","w"):pass
  clear_output()


In [ ]:
#@title Predict MHC-peptide structure

import py3Dmol
os.chdir("/content/")
if "MHCfold" not in dir():
  from timeit import default_timer as timer
  import sys
  sys.path.insert(0, '/content/MHCfold/')
  from MHCfold import *

mhcnet_dir_path = "/content/MHCfold/"
structure_model = os.path.join(mhcnet_dir_path, 'lean_model_for_classifier_new_padding2')
classificatione_model = os.path.join(mhcnet_dir_path, 'v7_date_5_9_2022')
error = False

if not os.path.exists(structure_model):
    print("Can't find trained structure module '{}', aborting.".format(structure_model), file=sys.stderr)
    error = True
  
if not os.path.exists(classificatione_model):
    print("Can't find trained classification model '{}', aborting.".format(classificatione_model), file=sys.stderr)
    error = True

output_flags = {"Structure": 1, "Binary_classification": 2, "Both": 3}

if not error:

    flags = f"-t {output_flags[prediction_output]}"
    if reconsrtuct_side_chains_using_modeller: flags+=" -m"
    if reconsrtuct_side_chains_using_modeller:
      !/content/compiled/MODELLER/bin/modpy.sh python MHCNet/MHCNet.py  $input_mhc -o $output_dir $flags
    else:
      !python MHCfold/MHCfold.py $input_mhc -o $output_dir $flags
    # run_nanonet(input_nb, nano_net_path, write_into_single_pdb_file, output_dir, reconsrtuct_side_chains_using_modeller, reconsrtuct_side_chains_using_scwrl)


def plot_structure(nb_name, pdb_path):
    with open(pdb_path) as ifile:
      predicted = "".join([x for x in ifile])
    r,g,b = 0,0,255
    print(f"\033[38;2;{r};{g};{b}m {nb_name} Predicted model \033[38;2;255;255;255m")
    view = py3Dmol.view(width=500, height=500)
    view.addModelsAsFrames(predicted)
    view.setStyle({'model': 0, 'chain': "A"}, {"cartoon": {'arrows':True, 'color': 'snow'}})
    view.setStyle({'model': 0, 'chain': "B"}, {"cartoon": {'arrows':True, 'color': 'turquoise'}})
    view.setStyle({'model': 0, 'chain': "C"}, {"cartoon": {'arrows':True, 'color': 'red'}})

    view.zoomTo()
    view.show()



file_ending = "mhcnet_backbone_cb.pdb" if not reconsrtuct_side_chains_using_modeller else "mhcnet_full_relaxed.pdb"
if visualize_results:
  print("Showing MHCNet predicted structures")
  for seq, nb_name in seq_iterator(input_mhc):
    plot_structure(nb_name, os.path.join(output_dir, f"{nb_name}_{file_ending}"))


/usr/local/lib/python3.8/dist-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(
1/1 [==============================] - 1s 648ms/step
/usr/local/lib/python3.8/dist-packages/Bio/PDB/Polypeptide.py:161: BiopythonDeprecationWarning: 'one_to_three' will be deprecated in a future release of Biopython in favor of 'Bio.PDB.Polypeptide.protein_letters_1to3'.
  warnings.warn(
MHCNet ended successfully, models are located in directory:'MHCNetResults', total time : 12.97925060199941.
Showing MHCNet predicted structures
 1A1O Predicted model 


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

 1A1M Predicted model 


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

 3VH8 Predicted model 


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

 4JFF Predicted model 


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

 6D2T Predicted model 


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
#@title Download results


!zip -FSr $output_dir".zip" $output_dir
files.download(f"{output_dir}.zip")

if save_to_google_drive == True and drive != None:
  uploaded = drive.CreateFile({'title': f"{output_dir}.zip"})
  uploaded.SetContentFile(f"{output_dir}.zip")
  uploaded.Upload()
  print(f"Uploaded {output_dir}.zip to Google Drive with ID {uploaded.get('id')}")

updating: MHCNetResults/ (stored 0%)
updating: MHCNetResults/classification_results (deflated 12%)
  adding: MHCNetResults/1A1O_mhcnet_backbone_cb.pdb (deflated 78%)
  adding: MHCNetResults/1A1M_mhcnet_backbone_cb.pdb (deflated 78%)
  adding: MHCNetResults/4JFF_mhcnet_backbone_cb.pdb (deflated 78%)
  adding: MHCNetResults/3VH8_mhcnet_backbone_cb.pdb (deflated 78%)
  adding: MHCNetResults/6D2T_mhcnet_backbone_cb.pdb (deflated 78%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>